In [1]:
!pip install dash==1.19.0  

!pip install jupyter_dash 

!pip install --upgrade plotly

     |████████████████████████████████| 75 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 51.4 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 52.8 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84011 sha256=9ed7409f755ed5f46966264a96d2913b511de01e3080bd53618a8f87dba7b198
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e7/27/2f/a9f9bf858843b5365bcb311a80f0b2379737caf4ec50b2ae3a
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527011 sha256=338e03ea329b169481c498b0d5b100348a657f9a304e23e83e8e2b8f15340b24
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e3/e6/0e/759ca580c568aad2

In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},],
                                value='ALL', placeholder="Select a Launch Site here",
                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),


                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                marks={0: '0', 2000: '2000', 4000: '4000', 6000: '6000', 8000: '8000', 10000: '10000'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', title='Total Successful Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        # return the outcomes piechart for a selected site
        fig = px.pie(filtered_df, values=[filtered_df['class'].value_counts()[0],filtered_df['class'].value_counts()[1]], 
        names=['Failed','Succeeded'], title='Total Successful Launches For Site '+entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,payload_slide):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_slide[0],payload_slide[1])]
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(payload_slide[0],payload_slide[1])]
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig

# Run the app
if __name__ == '__main__':
    


# Run the app
if __name__ == '__main__':
    #app.run_server()
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},],
                                value='ALL', placeholder="Select a Launch Site here",
                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),


                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                marks={0: '0', 2000: '2000', 4000: '4000', 6000: '6000', 8000: '8000', 10000: '10000'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', title='Total Successful Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        # return the outcomes piechart for a selected site
        fig = px.pie(filtered_df, values=[filtered_df['class'].value_counts()[0],filtered_df['class'].value_counts()[1]], 
        names=['Failed','Succeeded'], title='Total Successful Launches For Site '+entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,payload_slide):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_slide[0],payload_slide[1])]
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(payload_slide[0],payload_slide[1])]
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


FileNotFoundError: [Errno 2] No such file or directory: 'spacex_launch_dash.csv'